In [14]:
import requests
import json

In [27]:
def one_hop_message(curiea,typea,typeb,edgetype,reverse=False):
    query_graph = {
    "nodes": [
        {
            "id": "a",
            "type": typea,
            "curie": curiea
        },
        {
            "id": "b",
            "type": typeb
        }
    ],
    "edges": [
        {
            "id": "ab",
            "source_id": "a",
            "target_id": "b"
        }
    ]
    }
    if edgetype is not None:
        query_graph['edges'][0]['type'] = edgetype
        if reverse:
            query_graph['edges'][0]['source_id'] = 'b'
            query_graph['edges'][0]['target_id'] = 'a'
    message = {"message": {"query_graph": query_graph,
                          'knowledge_graph':{"nodes": [], "edges": [],}, 
                           'results':[{"node_bindings": [],"edge_bindings": []}]}}
    return message

In [84]:
def striderandfriends(message):
    url = 'http://robokop.renci.org:5781/query'
    response = requests.post(url,json=message)
    if not response.status_code == 200:
        print('strider:',response.status_code)
        return {}
    strider_answer = response.json()
    numanswers = len(strider_answer['results'])
    if (numanswers == 0) or ((numanswers == 1) and (len(strider_answer['results'][0]['node_bindings'])==0)):
        print('no answers')
        return {}
    #Strider for some reason doesn't return the query graph
    strider_answer['query_graph'] = message['message']['query_graph']
    message2 = {'message': strider_answer}
    weight_url = 'https://aragorn-ranker.renci.org/weight_correctness'
    response = requests.post(weight_url, json = message2)
    if not response.status_code == 200:
        print('weight:',response.status_code)
        return message2
    message3 = {'message': response.json()}
    score_url = 'https://aragorn-ranker.renci.org/score'
    response = requests.post(score_url, json = message3)
    if not response.status_code == 200:
        print('score:',response.status_code)
        return message3
    answercoalesce_url =  'https://answercoalesce.renci.org/coalesce'
    params = {'method':'graph'}
    message4 = {'message': response.json()}
    p_r = requests.post(url,json=message4,params=params)
    if not response.status_code == 200:
        print('ac:',response.status_code)
        return message4
    return p_r.json()

In [43]:
#Drugs that treat schizophrenia
#message = one_hop_message('MONDO:0005090','disease','chemical_substance','treats',reverse=True)
message= one_hop_message('MONDO:0005015','disease','chemical_substance','treats',reverse=True)
results = striderandfriends(message)
#print(results)

no answers


In [70]:
#Target for aspirin
message= one_hop_message('CHEBI:15365','chemical_substance','gene','related_to')
results = striderandfriends(message)

In [78]:
#genes that cause cystic fibrosis
message= one_hop_message('MONDO:0009061','disease','gene','causes',reverse=True)
results = striderandfriends(message)

no answers


In [85]:
#phenotypes of Charcot Marie Tooth
message = one_hop_message('MONDO:0015626','disease','phenotypic_feature','has_phenotype')
results = striderandfriends(message)

no answers


In [87]:
#What cells are in the brain?
message = one_hop_message('UBERON:0000955','anatomical_entity','cell','part_of',reverse=True)
results = striderandfriends(message)

no answers


In [88]:
#which genes enable glycolipid binding?
message = one_hop_message('GO:0051861','molecular_activity','gene','enabled_by')
results = striderandfriends(message)

no answers


In [89]:
#what part of the body does vasculitis affect?
message = one_hop_message('MONDO:0018882','disease','anatomical_entity','affects')
results = striderandfriends(message)

no answers
